# Model Free Analysis

Statistical inquiry into the aggregate behaviour of the *Wisconsin Sorting* & *NBack* Tasks.


---------
```
Zach Wolpe
zachcolinwolpe@gmail.com
17 July 2021
```
---------



# Executive Functions

The additional experiments are provided to gauge executive functions and computer literacy that may distinguish candidates when participating in the WCST & NBack Tasks.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re
import sys
sys.path.append('../process data/')
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
from encode_processed_data import encode_data

In [2]:
# with open('../data objects/batch_processing_object.pkl', 'rb') as file2:
#     bp = pickle.load(file2)

# ---- fetch data object ----x
with open('../data objects/batch_processing_object_with_encodings.pkl', 'rb') as file2:
    ed = pickle.load(file2)

   
ed.__dict__.keys()

dict_keys(['raw', 'fitts_summary_stats', 'corsi_summary_stats', 'navon_summary_stats', 'nback_summary_stats', 'demographics_plot', 'demographics'])

In [3]:
ed.describe_data()



        ------------------------------------------------------------------
            self.path            : raw data loc
            self.metadata        : mturk metadata
            self.mapping         : reference table
            self.data_times      : reference times table
            self.participants    : list of participant identifiers
            self.parti_code      : list of participant codes
            self.n               : total number of samples
            self.wcst_paths      : paths to wcst  raw data
            self.nback_paths     : paths to nback raw data
            self.corsi_paths     : paths to corsi raw data
            self.fitts_paths     : paths to fitts raw data
            self.navon_paths     : paths to navon raw data
            self.wcst_data       : wcst  dataframe
            self.nback_data      : nback dataframe
            self.corsi_data      : corsi dataframe
            self.fitts_data      : fitts dataframe
            self.navon_data    

In [4]:
ed.clean_data_info()



                WCST - Wisconsin Card Sorting Task                                                  DataFrame: ed.raw.wcst_date
            ---------------------------------------------------------------------------------------------------------------------------
            
                participant                     : key               : participant ID
                card_no                         : categorical       : the card shown
                correct_card                    : categorical       : the card that should be clicked of the top four on screen      
                correct_persevering             : categorical       : the card that would be clicked if the participant is persevering
                seq_no                          : numeric           : trial number
                rule                            : categorical       : matching rule  
                card_shape                      : categorical       : current card shape
                card_num

In [5]:
# ----- all categories descriptors -----x
cats_demographics   = ['gender_a', 'handedness_a', 'education_a', 'age_group']
cats_navon          = [('level_of_target', '')]

# ---- add numerical descriptors ----x
num_demographics = ['age_a','income_a', 'computer_hours_a', 'mean_reation_time_ms']
num_nback = [('block_number', ''), ('score', 'mean'), ('score', 'std'), ('status', 'mean'), ('status', 'std'), ('miss', 'mean'), 
            ('miss', 'std'), ('false_alarm', 'mean'), ('false_alarm', 'std'), ('reaction_time_ms', 'mean'), ('reaction_time_ms', 'std')]
num_navon = [('correct', 'mean'), ('correct',  'std'), ('too_slow', 'mean'), ('too_slow',  'std'), 
            ('reaction_time_ms', 'mean'), ('reaction_time_ms',  'std')]
num_corsi = [('highest_span',  'max'), ('status', 'mean'), ('status',  'std')]
num_fitts = [('delta', 'mean'), ('delta',  'std'), ('status', 'mean')]


# ---- user selected menus ---x

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.])

In [421]:
# ---- base plots ----x

# x < 25
# 25 <= x < 50
# 50 <= x < 75
# 75 <= x 

# df.iloc[df.index <25, :]

# ---- add trial number ----x
xx = []; df = ed.raw.wcst_data
[xx.append((i%100)+1) for i in range(df.shape[0])]
df['trial_no'] = xx 

# ---- status==1 --> correct


n_groups=4; n_groups=10
t = np.linspace(0,100,num=n_groups+1).tolist(); c=0

for tt in t[1:]:
    c +=1
    x = df.loc[df['trial_no'] < tt,].groupby(['participant', 'status']).agg({
    'participant':              ['count'],
    'reaction_time_ms':         ['mean', 'std'],
    'perseverance_error':       ['mean'],
    'not_perseverance_error':   ['mean']
    }).reset_index()
    x['percentages'] = x[('participant', 'count')]/tt
    x['trials']      = str(round(t[c-1])) + '-' + str(round(t[c]))
    x['trials_2']    = t[c]
    if c==1:    data=x
    else:       data=data.append(other=x)


data.head()

participant status participant reaction_time_ms               \
                           count             mean          std   
0    100934.0      1           9      2913.666667  1901.246170   
1    103322.0      1           8      1524.125000   714.072113   
2    103322.0      2           1      7644.000000          NaN   
3    107700.0      1           7      1640.857143   456.098465   
4    107700.0      2           2      3401.500000  1221.173411   

  perseverance_error not_perseverance_error percentages trials trials_2  
                mean                   mean                              
0                0.0                    0.0         0.9   0-10     10.0  
1                0.0                    0.0         0.8   0-10     10.0  
2                0.0                    1.0         0.1   0-10     10.0  
3                0.0                    0.0         0.7   0-10     10.0  
4                0.0                    1.0         0.2   0-10     10.0

In [494]:
 
# if x>0 --> perseverance_error > not_perseverance_error --> main error=perseverance_error
data['main_error'] = np.where(data['perseverance_error'] - data['not_perseverance_error'] > 0, 'perserverance errors', 'non perserverance errors')
data

participant status participant reaction_time_ms               \
                             count             mean          std   
0      100934.0      1           9      2913.666667  1901.246170   
1      103322.0      1           8      1524.125000   714.072113   
2      103322.0      2           1      7644.000000          NaN   
3      107700.0      1           7      1640.857143   456.098465   
4      107700.0      2           2      3401.500000  1221.173411   
..          ...    ...         ...              ...          ...   
665    996407.0      1          64      1189.859375   482.186484   
666    996407.0      2          35      1165.885714   468.933130   
667    998593.0      1          64      2898.187500  1283.695631   
668    998593.0      2          31      3669.064516  1744.578381   
669    998593.0      3           4      8994.750000  2010.500000   

    perseverance_error not_perseverance_error percentages  trials trials_2  \
                  mean                   mean                                
0             0.000000               0.000000        0.90    0-10     10.0   
1             0.000000               0.000000        0.80    0-10     10.0   
2             0.000000               1.000000        0.10    0-10     10.0   
3             0.000000               0.000000        0.70    0-10     10.0   
4             0.000000               1.000000        0.20    0-10     10.0   
..                 ...                    ...         ...     ...      ...   
665           0.000000               0.000000        0.64  90-100    100.0   
666           0.114286               0.885714        0.35  90-100    100.0   
667           0.000000               0.000000        0.64  90-100    100.0   
668           0.032258               0.967742        0.31  90-100    100.0   
669           0.250000               0.750000        0.04  90-100    100.0   

                   main_error main_error_color  
                                                
0    non perserverance errors            False  
1    non perserverance errors            False  
2    non perserverance errors            False  
3    non perserverance errors            False  
4    non perserverance errors            False  
..                        ...              ...  
665  non perserverance errors            False  
666  non perserverance errors            False  
667  non perserverance errors            False  
668  non perserverance errors            False  
669  non perserverance errors            False  

[6292 rows x 12 columns]

['steelblue',
 'steelblue',
 'steelblue',
 'steelblue',
 'steelblue',
 'steelblue',
 'steelblue',
 'steelblue',
 'steelblue',
 'steelblue']

In [510]:
import plotly.graph_objects as go
colours={'perserverance errors':'darkred', 'non perserverance errors': 'steelblue'}



traces = []
# for participant in data[('participant', '')].unique():
for participant in np.random.choice(data[('participant', '')].unique(), 5):
    df    = data.loc[(data[('participant', '')] == participant) & (data['status']==1), ['trials_2', 'percentages', 'main_error', 'reaction_time_ms']]
    trace = go.Scatter(x=df.trials_2, y=df.percentages, mode='lines+markers', marker=dict(
        size=df[('reaction_time_ms','mean')]/100,
        color=[colours[i] for i in df.main_error]
        ))
    traces.append(trace)
fig = go.Figure(data=traces)
fig.show()

In [390]:
traces = []
for participant in np.random.choice(data[('participant', '')].unique(), 5):
    df = data.loc[(data[('participant', '')] == participant) & (data['status']==1), ['trials_2', 'percentages', 'reaction_time_ms']]
    df['upper'] = df[('reaction_time_ms', 'mean')] + df[('reaction_time_ms', 'std')]
    df['lower'] = df[('reaction_time_ms', 'mean')] - df[('reaction_time_ms', 'std')]
    trace = go.Scatter(x=df[('reaction_time_ms', 'mean')], y=df.percentages, mode='markers',  
    error_x=dict(
            type='data',
            symmetric=False,
            array=df.upper,
            arrayminus=df.lower)
            )
    traces.append(trace)
fig = go.Figure(data=traces)
fig.show()

25

In [84]:
data

NameError: name 'data' is not defined

In [118]:
x

0             12     12  0.0      4258.416667  1919.036617   
925283.0              3      3  0.0      2615.666667  1391.819073   
929907.0              8      8  0.0      1338.750000   312.590901   
934851.0              7      7  0.0      3285.000000  1725.313691   
936870.0              4      4  0.0      1920.750000   467.094120   
937746.0              7      7  0.0      3878.714286  2540.092696   
938534.0              7      7  0.0      1831.714286  1620.488786   
941737.0             11     11  0.0      2520.181818   984.962316   
948158.0              6      6  0.0      2913.000000  1262.587977   
948299.0              4      4  0.0      2477.500000  1578.861510   
948779.0              5      5  0.0      3085.400000  1914.868742   
951022.0              6      6  0.0      1783.166667   867.047730   
951797.0             15     15  0.0      2055.866667   798.654392   
952799.0              6      6  0.0      1692.166667  1187.084397   
960412.0             10     10  0.0      2873.000000  2692.527313   
963191.0             19     19  0.0      3967.368421  2163.160425   
971229.0              4      4  0.0       973.000000   200.071654   
976061.0              5      5  0.0      1495.200000   565.777076   
977008.0             16     16  0.0      2058.187500  1433.205974   
978676.0              4      4  0.0      2302.750000  1518.696014   
979785.0              6      6  0.0      1202.333333   247.232414   
984020.0              9      9  0.0      1704.555556   723.798679   
987993.0             13     13  0.0      1602.000000   635.113113   
996407.0              7      7  0.0       994.857143   421.203604   
998593.0             13     13  0.0      4082.153846  2142.645438   

            perseverance_error      not_perseverance_error       
                          mean  std                   mean  std  
participant                                                      
100934.0                     0  0.0                      1  0.0  
103322.0                     0  0.0                      1  0.0  
107700.0                     0  0.0                      1  0.0  
117200.0                     0  0.0                      1  0.0  
117306.0                     0  0.0                      1  0.0  
120307.0                     0  0.0                      1  0.0  
122240.0                     0  0.0                      1  0.0  
127180.0                     0  0.0                      1  0.0  
130202.0                     0  0.0                      1  0.0  
130501.0                     0  0.0                      1  0.0  
135202.0                     0  0.0                      1  0.0  
136469.0                     0  0.0                      1  0.0  
139330.0                     0  0.0                      1  0.0  
140183.0                     0  0.0                      1  0.0  
141199.0                     0  0.0                      1  0.0  
142617.0                     0  0.0                      1  0.0  
152049.0                     0  0.0                      1  0.0  
153366.0                     0  0.0                      1  0.0  
157836.0                     0  0.0                      1  0.0  
160188.0                     0  0.0                      1  0.0  
165654.0                     0  0.0                      1  0.0  
172230.0                     0  0.0                      1  0.0  
174705.0                     0  0.0                      1  0.0  
181213.0                     0  0.0                      1  0.0  
183102.0                     0  0.0                      1  0.0  
187729.0                     0  0.0                      1  0.0  
188798.0                     0  0.0                      1  0.0  
190562.0                     0  0.0                      1  0.0  
193611.0                     0  0.0                      1  0.0  
195458.0                     0  0.0                      1  0.0  
196015.0                     0  0.0                      1  0.0  
203965.0                     0  0.0  

# Final Datasets

These curated datasets are now joined, pruned & transformed to produce the `final` set of datasets - on which we will conduct our anaylsis.


# Structure

#### wcst


In [12]:
ed.demographics.head()

,participant,age_a,gender_a,handedness_a,education_a,income_a,computer_hours_a,age_group,mean_reation_time_ms
0,816404.0,28.0,female,right,university,6.0,20.0,25-34,5502.000000
1,221478.0,25.0,female,right,graduate school,7.0,60.0,18-24,12353.428571
2,192208.0,29.0,male,right,university,6.0,5.0,25-34,8302.285714
3,803102.0,47.0,male,right,university,7.0,74.0,45-54,10340.857143
4,844810.0,32.0,female,right,university,6.0,22.0,25-34,6612.428571


In [185]:
# ed.demographics.head()
# ed.nback_summary_stats.head()
# ed.navon_summary_stats.head()
# ed.corsi_summary_stats.head()
# ed.fitts_summary_stats.head()

participant   delta             status          
                 mean         std   mean       std
0    100934.0   -5.70  186.607130    1.0  0.000000
1    103322.0 -203.05  174.308400    1.0  0.000000
2    107700.0 -342.65  345.637059    1.1  0.447214
3    117200.0  -85.05  156.117188    1.0  0.000000
4    117306.0 -141.80  167.808036    1.0  0.000000

In [152]:
# --- demographics dataset ---x
wcst = ed.raw.wcst_data[['participant', 'card_no', 'seq_no', 'rule', 'card_shape', 'card_number', 'card_colour', 'reaction_time_ms', 'status',
                                    'card_selected', 'error','perseverance_error', 'not_perseverance_error']]
wcst.set_index('participant').join(ed.demographics.set_index('participant'))


,card_no,seq_no,rule,card_shape,card_number,card_colour,reaction_time_ms,status,card_selected,error,perseverance_error,not_perseverance_error,age_a,gender_a,handedness_a,education_a,income_a,computer_hours_a,age_group,mean_reation_time_ms
participant,,,,,,,,,,,,,,,,,,,,
100934.0,52,1,shape,triangle,1,yellow,4567,1,1,0,0,0,28.0,male,right,university,7.0,25.0,25-34,11453.571429
100934.0,59,2,shape,triangle,3,red,4661,1,3,0,0,0,28.0,male,right,university,7.0,25.0,25-34,11453.571429
100934.0,23,3,shape,cross,2,red,1319,1,2,0,0,0,28.0,male,right,university,7.0,25.0,25-34,11453.571429
100934.0,52,4,shape,triangle,1,yellow,2336,1,1,0,0,0,28.0,male,right,university,7.0,25.0,25-34,11453.571429
100934.0,52,5,shape,triangle,1,yellow,6634,1,1,0,0,0,28.0,male,right,university,7.0,25.0,25-34,11453.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998593.0,61,2,color,triangle,4,blue,2489,1,4,0,0,0,25.0,female,right,graduate school,8.0,72.0,18-24,9284.714286
998593.0,14,3,color,circle,4,green,2193,1,4,0,0,0,25.0,female,right,graduate school,8.0,72.0,18-24,9284.714286
998593.0,14,4,color,circle,4,green,2310,1,4,0,0,0,25.0,female,right,graduate school,8.0,72.0,18-24,9284.714286


In [169]:
# --- demographics dataset ---x
# ed.nback_summary_stats
# ed.fitts_summary_stats

In [61]:
# ---- Scatter Plots ----x

# --- data ---x
df1 = ed.raw.wcst_data[['participant', 'seq_no', 'card_no', 'rule', 'card_shape', 'card_number',
                        'card_colour', 'reaction_time_ms',
                        'status', 'card_selected', 'error','perseverance_error', 'not_perseverance_error']]
df2 = ed.demographics[[ 'participant', 'age_a','gender_a','handedness_a','education_a', 'income_a', 
                        'computer_hours_a','age_group']]
df  = df1.set_index('participant').join(df2.set_index('participant'))


# --- scatter plot ---x
trace_1 = go.Scatter(x=df.computer_hours_a, y=df.age_a, mode='markers', marker=dict(size=12, color='steelblue', symbol='pentagon', line={'width':2}))
data    = [trace_1]
layout  = go.Layout(title='title', xaxis={'title':'age'}, yaxis={'title':'computer hours'}, hovermode='closest')
fig     = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='temp-plot.html')

'temp-plot.html'

In [66]:
# ---- line plot ----x
trace_1 = go.Scatter(x=df.seq_no, y=df.gender_a, mode='markers', marker=dict(size=5, color='darkblue', symbol='circle'))
trace_2 = go.Scatter(x=df.seq_no, y=df.education_a, mode='markers', marker=dict(size=5, color='darkblue', symbol='circle'))

data    = [trace_1, trace_2]
layout  = go.Layout(title='R Time', xaxis={'title':'age'}, yaxis={'title':'computer hours'})
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig)


'temp-plot.html'